# Análise Estatística ARC - Estresse (TSST-G) × Memória (ARC)
## Versão 8.0

Este notebook contém análises estatísticas para o projeto de pesquisa sobre os efeitos do estresse na memória, utilizando o paradigma TSST-G (Trier Social Stress Test - Group version) e medidas de organização semântica (ARC - Adjusted Ratio of Clustering).

In [1]:

def get_degrees_freedom(model_or_df, source=None):
    """Helper to extract degrees of freedom robustly."""
    try:
        if isinstance(model_or_df, pd.DataFrame): # Pingouin ANOVA table
            if source:
                if source in model_or_df["Source"].tolist():
                    df_effect = model_or_df.loc[model_or_df["Source"] == source, "DF"].iloc[0]
                else:
                    df_effect = "err" 
                if "Residual" in model_or_df["Source"].tolist():
                     df_error = model_or_df.loc[model_or_df["Source"] == "Residual", "DF"].iloc[0]
                else:
                     df_error = "err"
                df_effect_int = int(df_effect) if isinstance(df_effect, (int, float)) and not pd.isna(df_effect) else df_effect
                df_error_int = int(df_error) if isinstance(df_error, (int, float)) and not pd.isna(df_error) else df_error
                return df_effect_int, df_error_int
            else: 
                df_effect = model_or_df["DF"].iloc[0]
                df_error = model_or_df["DF"].iloc[-1]
                return int(df_effect), int(df_error)
        elif hasattr(model_or_df, "df_num") and hasattr(model_or_df, "df_den"): 
            return int(model_or_df.df_num), int(model_or_df.df_den)
        elif hasattr(model_or_df, "DF"): 
             return int(model_or_df["DF"].iloc[0])
    except (IndexError, TypeError, KeyError, ValueError):
        return "err", "err" 
    return "err", "err"

def format_p_value(p):
    """Formats p-value according to APA style."""
    if pd.isna(p):
        return "p = n.s."
    p = float(p)
    if p < 0.001:
        return "p < .001"
    elif p < 0.01:
        return f"p = {p:.3f}".replace("0.", ".")
    elif p < 0.05:
        return f"p = {p:.3f}".replace("0.", ".")
    else:
        return f"p = {p:.3f}".replace("0.", ".")

def format_stat(value):
    """Formats F or t value."""
    if pd.isna(value):
        return "N/A"
    return f"{float(value):.2f}"

def format_eta_squared(value):
    """Formats eta-squared."""
    if pd.isna(value):
        return "N/A"
    eta_val = float(value)
    if eta_val < 0.001:
         return "η²p < .001"
    return f"η²p = {eta_val:.3f}".replace("0.", ".")

def format_cohen_d(value):
    """Formats Cohen's d."""
    if pd.isna(value):
        return "N/A"
    return f"d = {float(value):.2f}"

def format_anova_apa_v5(aov_table, dv_name):
    """Formats 2x2x2 ANOVA results in APA style (v5)."""
    if aov_table is None or aov_table.empty:
        return f"Análise ANOVA para {dv_name} não pôde ser realizada."
    results = []
    source_order = [
        "Group", "Strategy", "nback_performance", 
        "Group * Strategy", "Group * nback_performance", "Strategy * nback_performance",
        "Group * Strategy * nback_performance"
    ]
    sources_in_table = [s for s in source_order if s in aov_table["Source"].tolist()]
    text = f"Uma ANOVA fatorial 2 (Grupo: Controle, Estressado) x 2 (Estratégia: Releitura, Prática de Lembrar) x 2 (Desempenho N-Back: Baixo, Alto) foi conduzida para examinar os efeitos na variável {dv_name}. "
    for source in sources_in_table:
        row = aov_table[aov_table["Source"] == source]
        if row.empty:
            continue
        f_val = row["F"].iloc[0]
        p_val = row["p-unc"].iloc[0]
        eta_sq = row["n2p"].iloc[0]
        df_effect, df_error = get_degrees_freedom(aov_table, source)
        if "*" in source:
            interaction_terms = source.split(" * ") 
            readable_terms = []
            for term in interaction_terms:
                if term == "nback_performance":
                    readable_terms.append("N-Back")
                elif term == "words_recalled":
                     readable_terms.append("Palavras Recordadas")
                else:
                    readable_terms.append(term)
            interaction_desc = " x ".join(readable_terms)
            effect_desc = f"a interação {interaction_desc}"
        else:
            if source == "nback_performance":
                readable_source = "N-Back"
            elif source == "words_recalled":
                 readable_source = "Palavras Recordadas"
            else:
                readable_source = source
            effect_desc = f"o efeito principal de {readable_source}"
        significance = "não foi significativo" if p_val >= 0.05 else "foi significativo"
        if df_effect == "err" or df_error == "err":
            f_string = f"*F*(gl_efeito?, gl_erro?) = {format_stat(f_val)}"
        else:
            f_string = f"*F*({df_effect}, {df_error}) = {format_stat(f_val)}"
        results.append(f"{effect_desc} {significance}, {f_string}, {format_p_value(p_val)}, {format_eta_squared(eta_sq)}")
    if len(results) == 0:
        text += "Nenhum efeito pôde ser analisado."
    elif len(results) == 1:
        text += results[0] + "."
    else:
        text += ", ".join(results[:-1]) + " e " + results[-1] + "."
    return text

def format_ttest_apa(ttest_table, comparison_desc, group1_desc=None, group2_desc=None, m1=None, sd1=None, m2=None, sd2=None):
    """Formats t-test results in APA style."""
    if ttest_table is None or ttest_table.empty:
        return f"Teste t para {comparison_desc} não pôde ser realizado."
    row = ttest_table.iloc[0]
    t_val = row["T"]
    p_val = row["p-val"]
    d_val = row["cohen-d"]
    df_val = get_degrees_freedom(ttest_table)
    test_type = "pareado" if row["paired"] else "de amostras independentes"
    alternative = row["alternative"]
    tail = "unilateral" if alternative != "two-sided" else "bilateral"
    significance = "não foi significativa" if p_val >= 0.05 else "foi significativa"
    if df_val == "err":
         t_string = f"*t*(gl?) = {format_stat(t_val)}"
    else:
         t_string = f"*t*({df_val}) = {format_stat(t_val)}"
    text = f"Um teste t {test_type} ({tail}) indicou que a diferença {comparison_desc} {significance}, {t_string}, {format_p_value(p_val)}, {format_cohen_d(d_val)}."
    if test_type == "de amostras independentes" and m1 is not None and sd1 is not None and m2 is not None and sd2 is not None and group1_desc and group2_desc:
        comparison_word = "similares"
        if significance == "foi significativa":
            comparison_word = "maiores" if m1 > m2 else "menores"
        text += f" O grupo {group1_desc} (*M* = {m1:.2f}, *DP* = {sd1:.2f}) apresentou escores {comparison_word} em comparação ao grupo {group2_desc} (*M* = {m2:.2f}, *DP* = {sd2:.2f})."
    elif test_type == "pareado" and m1 is not None and sd1 is not None and m2 is not None and sd2 is not None:
         text += f" Os escores mudaram de (*M* = {m1:.2f}, *DP* = {sd1:.2f}) para (*M* = {m2:.2f}, *DP* = {sd2:.2f})."
    return text

def format_mediation_apa(mediation_results):
    """Formats mediation analysis results in APA style."""
    if mediation_results is None or mediation_results.empty:
        return "Análise de mediação não pôde ser realizada."
    try:
        indirect_effect = mediation_results.loc[mediation_results["path"] == "Indirect", "coef"].iloc[0]
        indirect_pval = mediation_results.loc[mediation_results["path"] == "Indirect", "pval"].iloc[0]
        indirect_ci_lower = mediation_results.loc[mediation_results["path"] == "Indirect", "CI[2.5%]"]
        indirect_ci_upper = mediation_results.loc[mediation_results["path"] == "Indirect", "CI[97.5%]"]
        # Handle potential Series if multiple rows match (should not happen for 'Indirect')
        if isinstance(indirect_ci_lower, pd.Series):
            indirect_ci_lower = indirect_ci_lower.iloc[0]
        if isinstance(indirect_ci_upper, pd.Series):
            indirect_ci_upper = indirect_ci_upper.iloc[0]

        direct_effect = mediation_results.loc[mediation_results["path"] == "X -> Y", "coef"].iloc[0]
        direct_pval = mediation_results.loc[mediation_results["path"] == "X -> Y", "pval"].iloc[0]
        total_effect = mediation_results.loc[mediation_results["path"] == "Total", "coef"].iloc[0]
        total_pval = mediation_results.loc[mediation_results["path"] == "Total", "pval"].iloc[0]
        x_m_effect = mediation_results.loc[mediation_results["path"] == "X -> M", "coef"].iloc[0]
        x_m_pval = mediation_results.loc[mediation_results["path"] == "X -> M", "pval"].iloc[0]
        m_y_effect = mediation_results.loc[mediation_results["path"] == "M -> Y", "coef"].iloc[0]
        m_y_pval = mediation_results.loc[mediation_results["path"] == "M -> Y", "pval"].iloc[0]

        text = "Uma análise de mediação foi conduzida para investigar se o efeito da Estratégia (X) na Recordação Livre (Y) é mediado pela Organização Semântica (ARC, M), controlando pelo Grupo (covariável). "
        significance_xm = "não foi significativo" if x_m_pval >= 0.05 else "foi significativo"
        text += f"O efeito da Estratégia no ARC {significance_xm} (β = {x_m_effect:.3f}, {format_p_value(x_m_pval)}). "
        significance_my = "não foi significativo" if m_y_pval >= 0.05 else "foi significativo"
        text += f"O efeito do ARC na Recordação Livre, controlando pela Estratégia e Grupo, {significance_my} (β = {m_y_effect:.3f}, {format_p_value(m_y_pval)}). "
        significance_indirect = "não foi significativo" if indirect_pval >= 0.05 else "foi significativo"
        text += f"O efeito indireto da Estratégia na Recordação Livre através do ARC {significance_indirect} (Efeito Indireto = {indirect_effect:.3f}, IC 95% [{indirect_ci_lower:.3f}, {indirect_ci_upper:.3f}], {format_p_value(indirect_pval)}). "
        significance_direct = "não foi significativo" if direct_pval >= 0.05 else "foi significativo"
        text += f"O efeito direto da Estratégia na Recordação Livre, controlando pelo ARC e Grupo, {significance_direct} (β = {direct_effect:.3f}, {format_p_value(direct_pval)}). "
        significance_total = "não foi significativo" if total_pval >= 0.05 else "foi significativo"
        text += f"O efeito total da Estratégia na Recordação Livre {significance_total} (β = {total_effect:.3f}, {format_p_value(total_pval)})."
    except (IndexError, KeyError, TypeError) as e:
        return f"Erro ao formatar resultados da mediação: {e}. Verifique a tabela de resultados."
    return text

def add_analysis_to_excel(writer, sheet_name, df_results, intro_text, apa_text, img_path=None):
    """Adds analysis results (intro, table, APA text, image) to an Excel sheet."""
    workbook = writer.book
    try:
        if "Sheet1" in workbook.sheetnames and len(workbook["Sheet1"]._cells) == 0:
             del workbook["Sheet1"]
    except KeyError:
        pass 
    if sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]
        print(f"Aviso: Planilha 	{sheet_name}	 já existe. Sobrescrevendo conteúdo.")
        for row in worksheet.iter_rows():
            for cell in row:
                cell.value = None
    else:
        worksheet = workbook.create_sheet(sheet_name)
    current_row = 1
    if intro_text:
        worksheet.cell(row=current_row, column=1, value=intro_text)
        worksheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row + 2, end_column=10) 
        worksheet.cell(row=current_row, column=1).alignment = Alignment(wrap_text=True, vertical="top")
        worksheet.cell(row=current_row, column=1).font = Font(size=11, italic=True)
        current_row += 4 
    if apa_text:
        worksheet.cell(row=current_row, column=1, value=apa_text)
        worksheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row + 2, end_column=10) 
        worksheet.cell(row=current_row, column=1).alignment = Alignment(wrap_text=True, vertical="top")
        worksheet.cell(row=current_row, column=1).font = Font(size=11)
        current_row += 4 
    start_row_df = current_row 
    if df_results is not None and not df_results.empty:
        df_to_write = df_results.copy()
        if df_to_write.index.name is None or isinstance(df_to_write.index.name, int):
             df_to_write.index.name = "Index"
        elif df_to_write.index.name == "path": 
             df_to_write.index.name = "Caminho"
        for r_idx, row in enumerate(dataframe_to_rows(df_to_write, index=True, header=True), start_row_df):
            for c_idx, value in enumerate(row, 1):
                cell = worksheet.cell(row=r_idx, column=c_idx, value=value)
                if isinstance(value, (int, float)):
                    try:
                        cell.number_format = 	'0.000'	 
                    except (TypeError, ValueError):
                        pass 
                if r_idx == start_row_df: 
                    cell.font = Font(bold=True)
        current_row += len(df_to_write) + 2 
    else:
        worksheet.cell(row=start_row_df, column=1, value="(Nenhuma tabela de dados para esta análise)")
        current_row += 2
    img_anchor_row = current_row
    if img_path and os.path.exists(img_path):
        try:
            img = Image(img_path)
            worksheet.add_image(img, f"A{img_anchor_row}") 
        except Exception as e:
            print(f"Erro ao adicionar imagem {img_path} à planilha {sheet_name}: {e}")
            worksheet.cell(row=img_anchor_row, column=1, value=f"(Erro ao carregar imagem: {os.path.basename(img_path)})" )
    elif img_path:
        print(f"Aviso: Arquivo de imagem não encontrado para adicionar à planilha {sheet_name}: {img_path}")
        worksheet.cell(row=img_anchor_row, column=1, value=f"(Imagem não encontrada: {os.path.basename(img_path)})" )

def plot_descriptive_bar(df, dv, factors, filename):
    """Generates and saves a bar plot with error bars."""
    try:
        plt.figure(figsize=(8, 6))
        # Ensure factors are treated as categorical for plotting if they are not already
        # This is important if they were, for example, numerically coded earlier
        plot_df = df.copy()
        if factors[0] in plot_df.columns:
            plot_df[factors[0]] = plot_df[factors[0]].astype('category')
        if len(factors) > 1 and factors[1] in plot_df.columns:
            plot_df[factors[1]] = plot_df[factors[1]].astype('category')

        ax = sns.barplot(data=plot_df, x=factors[0], y=dv, hue=factors[1] if len(factors)>1 else None, 
                         errorbar='sd', capsize=.1, palette='viridis') # Added palette for better visuals
        
        title_text = f"Média de {dv} por { ' e '.join(factors) }"
        plt.title(title_text, fontsize=14)
        plt.ylabel(f"Média de {dv}", fontsize=12)
        plt.xlabel(factors[0], fontsize=12)
        if len(factors) > 1 and factors[1] in plot_df.columns:
            plt.legend(title=factors[1], title_fontsize='10', fontsize='9')
        plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for readability
        plt.tight_layout()
        plt.savefig(filename)
        plt.close() 
        print(f"Gráfico salvo: {filename}")
        return filename
    except Exception as e:
        print(f"Erro ao gerar gráfico {filename}: {e}")
        return None


In [4]:

import pandas as pd
import pingouin as pg
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment
from math import sqrt

data_path = r"D:\PhD_Jupyter\files\data\clean_data\measures_phd.csv" # Assuming data is uploaded here
excel_path = r"D:\PhD_Jupyter\files\data\analysis\resultados_analise_v7.xlsx"
image_dir = r"D:\PhD_Jupyter\files\data\analysis\images"

os.makedirs(image_dir, exist_ok=True)

try:
    df = pd.read_csv(data_path)
    print("Dados carregados com sucesso.")
    print(df.head())
    print(f"Shape dos dados: {df.shape}")
    print("\nInformações do DataFrame:") 
    df.info()
except FileNotFoundError:
    print(f"Erro: Arquivo de dados não encontrado em {data_path}")
    df = None 

if df is not None:
    if 'Group' in df.columns:
        df['Group'] = df['Group'].astype('category')
    if 'Strategy' in df.columns:
        df['Strategy'] = df['Strategy'].astype('category')
    if 'nback_score' in df.columns:
        median_nback = df['nback_score'].median()
        df['nback_performance'] = pd.cut(df['nback_score'], 
                                         bins=[-np.inf, median_nback, np.inf], 
                                         labels=['Baixo', 'Alto'], 
                                         right=True) 
        print(f"\nMediana do N-Back Score: {median_nback}") 
        print("Distribuição da performance no N-Back:")
        print(df['nback_performance'].value_counts())
    else:
        print("Aviso: Coluna 'nback_score' não encontrada. Categoria 'nback_performance' não criada.")
        df['nback_performance'] = 'Desconhecido' 
    df['nback_performance'] = df['nback_performance'].astype('category')
    print("\nTipos de dados após preparação inicial:") 
    print(df.dtypes)
else:
    print("Processamento de dados não realizado devido a erro no carregamento.")


Dados carregados com sucesso.
    ID_codes  Strategy  Group  JOSE  words_recalled  categorical_access  \
0  100614101         1      0     1              25                   7   
1  103484010         0      1     0              16                   6   
2  106977011         0      1     1              23                   7   
3  112327001         0      0     1              17                   7   
4  126855010         0      1     0              23                   8   

    IPCA   R  expectedR  maxR  ...  recog_target_rt  recog_unrelated_score  \
0  3.571  12      2.800    18  ...            0.764                     20   
1  2.667   6      2.250    10  ...            1.004                     20   
2  3.286   7      2.783    16  ...            1.055                     19   
3  2.429   7      2.000    10  ...            1.246                     19   
4  2.875  10      2.435    15  ...            1.219                     18   

   recog_unrelated_rt  recog_related_score  recog_

## Estatísticas Descritivas

Esta seção apresenta estatísticas descritivas para as principais variáveis dependentes (ARC, Palavras Recordadas, BAI, PANAS), separadas por grupo experimental e estratégia de estudo.

In [5]:

if df is not None:
    print("\n--- Estatísticas Descritivas ---") 
    desc_arc = df.groupby(['Group', 'Strategy'], observed=False)['ARC'].agg(['count', 'mean', 'std', 'min', 'max']).reset_index()
    desc_words = df.groupby(['Group', 'Strategy'], observed=False)['words_recalled'].agg(['count', 'mean', 'std', 'min', 'max']).reset_index()
    desc_bai = df.groupby(['Group', 'Strategy'], observed=False)['BAIdelta'].agg(['count', 'mean', 'std', 'min', 'max']).reset_index()
    desc_panas = df.groupby(['Group', 'Strategy'], observed=False)['PANASdelta'].agg(['count', 'mean', 'std', 'min', 'max']).reset_index()

    print("\nDescritivas para ARC:") 
    print(desc_arc)
    print("\nDescritivas para Palavras Recordadas:") 
    print(desc_words)
    print("\nDescritivas para Delta BAI:") 
    print(desc_bai)
    print("\nDescritivas para Delta PANAS:") 
    print(desc_panas)

    img_path_arc_int = plot_descriptive_bar(df, 'ARC', ['Group', 'Strategy'], os.path.join(image_dir, 'desc_arc_interaction.png'))
    img_path_words_int = plot_descriptive_bar(df, 'words_recalled', ['Group', 'Strategy'], os.path.join(image_dir, 'desc_words_interaction.png'))
    img_path_bai_int = plot_descriptive_bar(df, 'BAIdelta', ['Group', 'Strategy'], os.path.join(image_dir, 'desc_bai_delta_interaction.png'))
    img_path_panas_int = plot_descriptive_bar(df, 'PANASdelta', ['Group', 'Strategy'], os.path.join(image_dir, 'desc_panas_delta_interaction.png'))

    intro_desc = "Esta seção apresenta as estatísticas descritivas (contagem, média, desvio padrão, mínimo, máximo) para as principais variáveis dependentes (ARC, Palavras Recordadas, Delta BAI, Delta PANAS), separadas pelos grupos experimentais (Controle vs. Estressado) e pela estratégia de estudo utilizada (Releitura vs. Prática de Lembrar)." 
    apa_desc = "As médias e desvios padrões para cada condição são apresentados na tabela e no gráfico."

    try:
        writer = pd.ExcelWriter(excel_path, engine='openpyxl') 
        print(f"ExcelWriter inicializado para {excel_path}")
        add_analysis_to_excel(writer, 'Desc_ARC', desc_arc, intro_desc, apa_desc, img_path_arc_int)
        add_analysis_to_excel(writer, 'Desc_Palavras', desc_words, intro_desc, apa_desc, img_path_words_int)
        add_analysis_to_excel(writer, 'Desc_BAI', desc_bai, intro_desc, apa_desc, img_path_bai_int)
        add_analysis_to_excel(writer, 'Desc_PANAS', desc_panas, intro_desc, apa_desc, img_path_panas_int)
        print(f"Estatísticas descritivas adicionadas ao Excel: {excel_path}")
    except NameError: 
        print("Erro: Variável 'excel_path' não definida. O Excel Writer não pode ser inicializado.")
        writer = None
    except Exception as e:
        print(f"Erro ao processar estatísticas descritivas ou iniciar Excel: {e}")
        writer = None 
else:
    print("Dados não carregados. Estatísticas descritivas não calculadas.")
    writer = None 



--- Estatísticas Descritivas ---

Descritivas para ARC:
  Group Strategy  count      mean       std    min    max
0     0        0     31  0.610645  0.192293  0.284  1.000
1     0        1     29  0.652276  0.209314  0.133  0.952
2     1        0     34  0.557853  0.244050 -0.038  1.000
3     1        1     34  0.633794  0.293612 -0.400  1.000

Descritivas para Palavras Recordadas:
  Group Strategy  count       mean       std  min  max
0     0        0     31  19.967742  6.896781    9   36
1     0        1     29  25.655172  7.001407   11   36
2     1        0     34  20.382353  8.889949    6   39
3     1        1     34  26.205882  8.831004    7   38

Descritivas para Delta BAI:
  Group Strategy  count      mean       std  min  max
0     0        0     31 -2.193548  5.612007  -27    5
1     0        1     29 -2.793103  4.995565  -22    3
2     1        0     34  1.647059  4.220314   -7   14
3     1        1     34  2.029412  5.578559  -15   16

Descritivas para Delta PANAS:
  Group S

## Verificação da Manipulação de Estresse

Esta seção verifica se a manipulação de estresse foi eficaz, comparando as medidas de ansiedade (BAI) e afeto negativo (PANAS) antes e depois da tarefa estressora.

In [6]:

if df is not None and writer is not None:
    print("\n--- Verificação da Manipulação de Estresse ---") 
    control_group = df[df['Group'] == 'Control']
    stress_group = df[df['Group'] == 'Stress']

    ttest_bai_stress = pg.ttest(stress_group['BAIpre'], stress_group['BAIpos'], paired=True)
    m_pre_bai_s = stress_group['BAIpre'].mean()
    sd_pre_bai_s = stress_group['BAIpre'].std()
    m_pos_bai_s = stress_group['BAIpos'].mean()
    sd_pos_bai_s = stress_group['BAIpos'].std()
    apa_bai_stress = format_ttest_apa(ttest_bai_stress, "entre BAI pré e pós-estressor no grupo Estresse", 
                                      m1=m_pre_bai_s, sd1=sd_pre_bai_s, m2=m_pos_bai_s, sd2=sd_pos_bai_s)
    print("Teste t pareado BAI (Grupo Estresse):")
    print(ttest_bai_stress)
    print(apa_bai_stress)

    ttest_panas_stress = pg.ttest(stress_group['PANASpre'], stress_group['PANASpost'], paired=True)
    m_pre_panas_s = stress_group['PANASpre'].mean()
    sd_pre_panas_s = stress_group['PANASpre'].std()
    m_pos_panas_s = stress_group['PANASpost'].mean()
    sd_pos_panas_s = stress_group['PANASpost'].std()
    apa_panas_stress = format_ttest_apa(ttest_panas_stress, "entre PANAS Negativo pré e pós-estressor no grupo Estresse",
                                        m1=m_pre_panas_s, sd1=sd_pre_panas_s, m2=m_pos_panas_s, sd2=sd_pos_panas_s)
    print("\nTeste t pareado PANAS (Grupo Estresse):") 
    print(ttest_panas_stress)
    print(apa_panas_stress)

    ttest_bai_delta = pg.ttest(control_group['BAIdelta'], stress_group['BAIdelta'], correction=True) 
    m_delta_bai_c = control_group['BAIdelta'].mean()
    sd_delta_bai_c = control_group['BAIdelta'].std()
    m_delta_bai_s = stress_group['BAIdelta'].mean()
    sd_delta_bai_s = stress_group['BAIdelta'].std()
    apa_bai_delta = format_ttest_apa(ttest_bai_delta, "na variação do BAI (Delta) entre os grupos Controle e Estresse",
                                     group1_desc="Controle", m1=m_delta_bai_c, sd1=sd_delta_bai_c, 
                                     group2_desc="Estresse", m2=m_delta_bai_s, sd2=sd_delta_bai_s)
    print("\nTeste t independente Delta BAI (Controle vs Estresse):") 
    print(ttest_bai_delta)
    print(apa_bai_delta)

    ttest_panas_delta = pg.ttest(control_group['PANASdelta'], stress_group['PANASdelta'], correction=True)
    m_delta_panas_c = control_group['PANASdelta'].mean()
    sd_delta_panas_c = control_group['PANASdelta'].std()
    m_delta_panas_s = stress_group['PANASdelta'].mean()
    sd_delta_panas_s = stress_group['PANASdelta'].std()
    apa_panas_delta = format_ttest_apa(ttest_panas_delta, "na variação do PANAS Negativo (Delta) entre os grupos Controle e Estresse",
                                       group1_desc="Controle", m1=m_delta_panas_c, sd1=sd_delta_panas_c, 
                                       group2_desc="Estresse", m2=m_delta_panas_s, sd2=sd_delta_panas_s)
    print("\nTeste t independente Delta PANAS (Controle vs Estresse):") 
    print(ttest_panas_delta)
    print(apa_panas_delta)

    intro_manip_bai = "Esta análise verifica se a tarefa estressora (TSST-G) aumentou a ansiedade (BAI) no grupo Estresse (teste t pareado) e se a variação na ansiedade (Delta BAI) difere entre os grupos Controle e Estresse (teste t independente)." 
    intro_manip_panas = "Similarmente à análise do BAI, esta seção verifica se a manipulação de estresse afetou os escores de afeto negativo (PANAS Negativo). Testes t comparam os escores pré e pós-tarefa dentro do grupo Estresse e a variação (Delta) entre os grupos."
    bai_stats = df.groupby('Group', observed=False)[['BAIpre', 'BAIpos']].agg(['mean', 'std'])
    panas_stats = df.groupby('Group', observed=False)[['PANASpre', 'PANASpost']].agg(['mean', 'std'])
    apa_bai_full = f"{apa_bai_stress} {apa_bai_delta}"
    apa_panas_full = f"{apa_panas_stress} {apa_panas_delta}"
    manip_check_summary = pd.concat([
        ttest_bai_stress.assign(Test="BAI Pareado (Estresse)"),
        ttest_panas_stress.assign(Test="PANAS Pareado (Estresse)"),
        ttest_bai_delta.assign(Test="Delta BAI Indep (C vs E)"),
        ttest_panas_delta.assign(Test="Delta PANAS Indep (C vs E)")
    ], ignore_index=True)
    add_analysis_to_excel(writer, 'Manip_Check_BAI', ttest_bai_stress.join(ttest_bai_delta, lsuffix='_paired', rsuffix='_indep'), intro_manip_bai, apa_bai_full)
    add_analysis_to_excel(writer, 'Manip_Check_PANAS', ttest_panas_stress.join(ttest_panas_delta, lsuffix='_paired', rsuffix='_indep'), intro_manip_panas, apa_panas_full)
    print("Resultados da verificação de manipulação adicionados ao Excel.")
else:
    print("Dados ou Excel Writer não disponíveis. Verificação de manipulação não realizada.")



--- Verificação da Manipulação de Estresse ---


AssertionError: y cannot be an empty list or array.

## ANOVA Fatorial 2x2x2

Esta seção apresenta os resultados da ANOVA fatorial 2 (Grupo: Controle, Estressado) x 2 (Estratégia: Releitura, Prática de Lembrar) x 2 (Desempenho N-Back: Baixo, Alto) para as variáveis ARC e Palavras Recordadas.

In [ ]:

if df is not None and writer is not None:
    print("\n--- ANOVA Fatorial 2x2x2 (Grupo x Estratégia x N-Back Performance) ---") 
    df['Group'] = df['Group'].astype('category')
    df['Strategy'] = df['Strategy'].astype('category')
    df['nback_performance'] = df['nback_performance'].astype('category')
    print("\nANOVA 2x2x2 para ARC:") 
    try:
        anova_arc = pg.anova(data=df, dv='ARC', between=['Group', 'Strategy', 'nback_performance'], detailed=True)
        print(anova_arc)
        apa_anova_arc = format_anova_apa_v5(anova_arc, "ARC")
        print("\nTexto APA para ANOVA ARC:") 
        print(apa_anova_arc)
        img_path_anova_arc = os.path.join(image_dir, 'anova_arc_2x2x2_interaction.png')
        try:
            g = sns.catplot(data=df, x='Group', y='ARC', hue='Strategy', col='nback_performance', 
                            kind='bar', errorbar='sd', capsize=.1, height=4, aspect=1, palette='viridis')
            g.fig.suptitle('Interação Grupo x Estratégia x N-Back para ARC', y=1.03)
            g.set_axis_labels("Grupo", "Média ARC")
            g.set_titles("N-Back: {col_name}")
            plt.tight_layout()
            plt.savefig(img_path_anova_arc)
            plt.close(g.fig)
            print(f"Gráfico da ANOVA ARC salvo: {img_path_anova_arc}")
        except Exception as plot_e:
            print(f"Erro ao gerar gráfico da ANOVA ARC: {plot_e}")
            img_path_anova_arc = None
        intro_anova_arc = "Esta análise investiga os efeitos principais e de interação do Grupo (Controle vs. Estresse), Estratégia de Estudo (Releitura vs. Prática de Lembrar) e Desempenho na Tarefa N-Back (Baixo vs. Alto) na medida de organização semântica da recordação (ARC), utilizando uma ANOVA fatorial 2x2x2."
        add_analysis_to_excel(writer, 'ANOVA_ARC_2x2x2', anova_arc, intro_anova_arc, apa_anova_arc, img_path_anova_arc)
        print("Resultados da ANOVA 2x2x2 para ARC adicionados ao Excel.")
    except Exception as e:
        print(f"Erro ao calcular ANOVA 2x2x2 para ARC: {e}")
        anova_arc = None 
        apa_anova_arc = f"Erro ao calcular ANOVA 2x2x2 para ARC: {e}"
        add_analysis_to_excel(writer, 'ANOVA_ARC_2x2x2', None, intro_anova_arc, apa_anova_arc)

    print("\nANOVA 2x2x2 para Palavras Recordadas:") 
    try:
        anova_words = pg.anova(data=df, dv='words_recalled', between=['Group', 'Strategy', 'nback_performance'], detailed=True)
        print(anova_words)
        apa_anova_words = format_anova_apa_v5(anova_words, "Palavras Recordadas")
        print("\nTexto APA para ANOVA Palavras Recordadas:") 
        print(apa_anova_words)
        img_path_anova_words = os.path.join(image_dir, 'anova_words_2x2x2_interaction.png')
        try:
            g = sns.catplot(data=df, x='Group', y='words_recalled', hue='Strategy', col='nback_performance', 
                            kind='bar', errorbar='sd', capsize=.1, height=4, aspect=1, palette='viridis')
            g.fig.suptitle('Interação Grupo x Estratégia x N-Back para Palavras Recordadas', y=1.03)
            g.set_axis_labels("Grupo", "Média Palavras Recordadas")
            g.set_titles("N-Back: {col_name}")
            plt.tight_layout()
            plt.savefig(img_path_anova_words)
            plt.close(g.fig)
            print(f"Gráfico da ANOVA Palavras Recordadas salvo: {img_path_anova_words}")
        except Exception as plot_e:
            print(f"Erro ao gerar gráfico da ANOVA Palavras Recordadas: {plot_e}")
            img_path_anova_words = None
        intro_anova_words = "Esta análise investiga os efeitos principais e de interação do Grupo (Controle vs. Estresse), Estratégia de Estudo (Releitura vs. Prática de Lembrar) e Desempenho na Tarefa N-Back (Baixo vs. Alto) no número total de palavras recordadas, utilizando uma ANOVA fatorial 2x2x2."
        add_analysis_to_excel(writer, 'ANOVA_Words_2x2x2', anova_words, intro_anova_words, apa_anova_words, img_path_anova_words)
        print("Resultados da ANOVA 2x2x2 para Palavras Recordadas adicionados ao Excel.")
    except Exception as e:
        print(f"Erro ao calcular ANOVA 2x2x2 para Palavras Recordadas: {e}")
        anova_words = None 
        apa_anova_words = f"Erro ao calcular ANOVA 2x2x2 para Palavras Recordadas: {e}"
        add_analysis_to_excel(writer, 'ANOVA_Words_2x2x2', None, intro_anova_words, apa_anova_words)
else:
    print("Dados ou Excel Writer não disponíveis. ANOVA 2x2x2 não realizada.")
    anova_arc = None 
    anova_words = None


## Análise de Mediação

Esta seção investiga se o efeito da Estratégia de Estudo na Recordação Livre é mediado pela Organização Semântica (ARC), controlando pelo Grupo Experimental.

In [ ]:

if df is not None and writer is not None:
    print("\n--- Análise de Mediação ---") 
    df_mediation = df.copy()
    df_mediation['Strategy_code'] = df_mediation['Strategy'].apply(lambda x: 1 if x == 'Test' else 0) 
    df_mediation['Group_code'] = df_mediation['Group'].apply(lambda x: 1 if x == 'Stress' else 0) 
    try:
        mediation_results = pg.mediation_analysis(data=df_mediation, x='Strategy_code', m='ARC', y='words_recalled', 
                                                covar=['Group_code'], alpha=0.05, seed=42, n_boot=1000)
        print(mediation_results)
        apa_mediation = format_mediation_apa(mediation_results)
        print("\nTexto APA para Análise de Mediação:") 
        print(apa_mediation)
        intro_mediation = "Esta análise testa a hipótese de que o efeito da Estratégia de Estudo (X: Prática de Lembrar vs. Releitura) no número de Palavras Recordadas (Y) é mediado pela Organização Semântica (M: ARC), controlando pelo Grupo Experimental (Covariável: Controle vs. Estresse). Os resultados indicam os efeitos direto, indireto e total."
        add_analysis_to_excel(writer, 'Mediacao_ARC', mediation_results, intro_mediation, apa_mediation)
        print("Resultados da análise de mediação adicionados ao Excel.")
    except Exception as e:
        print(f"Erro ao calcular Análise de Mediação: {e}")
        mediation_results = None
        apa_mediation = f"Erro ao calcular Análise de Mediação: {e}"
        add_analysis_to_excel(writer, 'Mediacao_ARC', None, intro_mediation, apa_mediation)
else:
    print("Dados ou Excel Writer não disponíveis. Análise de Mediação não realizada.")
    mediation_results = None


## Análise da Hipótese e Efeitos de Interação

Esta seção discute a hipótese principal do estudo à luz dos resultados da ANOVA 2x2x2, focando na interação entre Grupo, Estratégia e Desempenho N-Back.

In [ ]:

if df is not None and writer is not None and 'anova_arc' in locals() and anova_arc is not None and 'anova_words' in locals() and anova_words is not None:
    print("\n--- Análise da Hipótese e Efeitos de Interação ---") 
    try:
        interaction_source_arc = "Group * Strategy * nback_performance"
        interaction_source_words = "Group * Strategy * nback_performance"
        interaction_p_arc = np.nan
        if interaction_source_arc in anova_arc["Source"].tolist():
            interaction_p_arc = anova_arc.loc[anova_arc["Source"] == interaction_source_arc, "p-unc"].iloc[0]
            print(f"Interação Grupo x Estratégia x N-Back para ARC: p = {interaction_p_arc:.4f}")
        else:
            print(f"Aviso: Interação '{interaction_source_arc}' não encontrada na tabela ANOVA ARC.")
        interaction_p_words = np.nan
        if interaction_source_words in anova_words["Source"].tolist():
            interaction_p_words = anova_words.loc[anova_words["Source"] == interaction_source_words, "p-unc"].iloc[0]
            print(f"Interação Grupo x Estratégia x N-Back para Palavras Recordadas: p = {interaction_p_words:.4f}")
        else:
             print(f"Aviso: Interação '{interaction_source_words}' não encontrada na tabela ANOVA Palavras.")

        print("\nAnálise da Hipótese Principal:")
        print("A hipótese central do estudo era que o estresse prejudicaria a memória de forma mais acentuada")
        print("no grupo que utilizou a Prática de Lembrar em comparação com a Releitura.")
        print("Isso seria refletido em uma interação significativa entre Grupo e Estratégia nas")
        print("medidas de memória (ARC e Palavras Recordadas).")
        print("\nResultados da ANOVA 2x2x2:")
        print(f"- Para ARC: A significância da interação tripla (Grupo x Estratégia x N-Back) foi p = {format_p_value(interaction_p_arc)}.")
        print(f"- Para Palavras Recordadas: A significância da interação tripla (Grupo x Estratégia x N-Back) foi p = {format_p_value(interaction_p_words)}.")
        print("\nConclusão Preliminar:")
        print("Com base nos resultados da ANOVA 2x2x2, avalie se a interação tripla (ou interações de duas vias relevantes como Grupo*Estratégia)")
        print("foi estatisticamente significativa (p < 0.05). Se não for significativa, não há evidência")
        print("estatística (neste nível de análise) para suportar a hipótese de que o efeito do estresse na memória")
        print("depende da estratégia de estudo utilizada, considerando também o desempenho cognitivo (N-Back).")
        print("Se alguma interação for significativa, análises post-hoc seriam necessárias para detalhar")
        print("as diferenças específicas entre as condições.")
        interpretation_text = (
            "Análise da Hipótese Principal:\n" + 
            "A hipótese central do estudo era que o estresse prejudicaria a memória de forma mais acentuada \n" + 
            "no grupo que utilizou a Prática de Lembrar em comparação com a Releitura. \n" + 
            "Isso seria refletido em uma interação significativa entre Grupo e Estratégia nas \n" + 
            "medidas de memória (ARC e Palavras Recordadas).\n\n" + 
            "Resultados da ANOVA 2x2x2:\n" + 
            f"- Para ARC: A significância da interação tripla (Grupo x Estratégia x N-Back) foi p = {format_p_value(interaction_p_arc)}.\n" + 
            f"- Para Palavras Recordadas: A significância da interação tripla (Grupo x Estratégia x N-Back) foi p = {format_p_value(interaction_p_words)}.\n\n" + 
            "Conclusão Preliminar:\n" + 
            "Com base nos resultados da ANOVA 2x2x2, avalie se a interação tripla (ou interações de duas vias relevantes como Grupo*Estratégia) \n" + 
            "foi estatisticamente significativa (p < 0.05). Se não for significativa, não há evidência \n" + 
            "estatística (neste nível de análise) para suportar a hipótese de que o efeito do estresse na memória \n" + 
            "depende da estratégia de estudo utilizada, considerando também o desempenho cognitivo (N-Back).\n" + 
            "Se alguma interação for significativa, análises post-hoc seriam necessárias para detalhar \n" + 
            "as diferenças específicas entre as condições."
        )
        intro_hypothesis = "Esta seção discute a hipótese principal do estudo à luz dos resultados da ANOVA 2x2x2, focando na interação entre Grupo, Estratégia e Desempenho N-Back para as variáveis ARC e Palavras Recordadas."
        add_analysis_to_excel(writer, 'Hipótese_Interacao', None, intro_hypothesis, interpretation_text)
        print("Análise da hipótese adicionada ao Excel.")
    except (KeyError, IndexError, TypeError) as e:
        print(f"Erro ao interpretar resultados da ANOVA para hipótese: {e}")
        error_text = (
            "Erro ao acessar resultados da ANOVA para análise da hipótese.\n" + 
            "Verifique se as ANOVAs foram executadas corretamente e se as variáveis 'anova_arc' e 'anova_words' contêm os resultados esperados.\n" + 
            f"Erro específico: {e}"
        )
        print(error_text)
        add_analysis_to_excel(writer, 'Hipótese_Interacao', None, "Erro na Análise da Hipótese", error_text)
else:
    print("Dados, Excel Writer ou resultados da ANOVA não disponíveis. Análise da hipótese não realizada.")


## Salvar Resultados em Excel

Esta seção salva todos os resultados das análises em um arquivo Excel, com abas separadas para cada análise.

In [ ]:

if writer is not None:
    try:
        writer.close() 
        print(f"Arquivo Excel final salvo com sucesso: {excel_path}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo Excel final: {e}")
else:
    print("Excel Writer não foi inicializado. Arquivo Excel não salvo.")
